In [179]:
# for Exited, 1 is not-exited, -1 is exited
# for gender, male is 0 and female is 1


#Consider creating a combined column for exited and active member
    #-2 if exited and active, -1 if exited and inactive, 1 if stayed and inactive, 2 if stayed and active

#clean outlier data from monthly salary (scale or remove?)

#need to do either undersampling or oversampling, about 8000 users stayed while only 2000 left

#standardscaler or create new features from the features (salary, balance, age, credit score, tenure)

# separate data by region? looks like most data is from france. Much less likely for french customers to leave.
# i.e. the leave data for all three countries is similar but the stay data is much higher in france.

#MY ASSUMPTIONS:
#The data was collected at two points in a year. First point gave balance, salary, etc., then later that year they checked if that person had left or stayed
#Balance may still exist if a person left the bank because it represents the balance they had before leaving
#User is considered an active user if they are still using the bank services in some way. its possible for an active member to leave. its possible for an inactive member to stay.

#is profitable idea: check a 1 for yes and -1 for no, based on iff the user is both an active member AND has a positive balance

In [180]:
import pandas as pd
import numpy as np
from pathlib import Path

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC


In [181]:

starter_df = pd.read_csv(Path("c://users/ajcth/documents/github/bank_churn_project/Resources/BankChurners.csv"))
starter_df.sample(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9537,9538,15780409,Egobudike,783,France,Male,40,6,0.00,2,1,0,109742.55,0
1683,1684,15709387,Obiajulu,711,France,Male,52,5,0.00,1,1,1,159808.95,0
8131,8132,15803352,Scott,613,Germany,Male,33,3,155736.42,2,1,1,57751.21,0
2978,2979,15595917,Mackay,580,France,Female,35,1,102097.33,1,0,1,168285.85,0
8652,8653,15787204,Howe,774,Spain,Female,43,1,110646.54,1,0,0,108804.28,0
7186,7187,15565996,Arnold,653,France,Male,44,8,0.00,2,1,1,154639.72,0
7548,7549,15658504,Chiawuotu,584,Germany,Female,62,9,137727.34,2,0,1,121102.90,0
2558,2559,15727021,Obialo,727,Germany,Female,30,8,119027.28,2,1,1,137903.54,0
1625,1626,15630321,Hu,680,France,Male,44,3,0.00,2,1,0,86935.08,0
4452,4453,15704788,Krawczyk,812,Spain,Female,49,8,66079.45,2,0,0,91556.57,1


In [182]:
#Convert the values of the Gender column from "M" or "F" to "0" or "1"
starter_df = starter_df.replace({'Gender' : {'Male': 0, 'Female': 1}})
#Make HasCrCard a -1 if they don't, so it reflects thats a negative
starter_df = starter_df.replace({'HasCrCard' : {1: -1, 0 : 1}})
#Make IsActiveMember a -1 if they arent, so it reflects thats a negative
starter_df = starter_df.replace({'Exited' : {0: -1}})
#Make IsActiveMember a -1 if they arent, so it reflects thats a negative
starter_df = starter_df.replace({'IsActiveMember' : {0: -1}})
#Drop the useless columns
starter_df = starter_df.drop(columns=['RowNumber', 'CustomerId', 'Surname'])
#Drop Nan's
starter_df.dropna(inplace=True)

In [183]:
starter_df.sample(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
5972,548,Germany,0,49,9,108437.89,1,1,-1,127022.87,1
9482,707,France,0,40,1,0.00,2,-1,-1,14090.40,1
1886,667,Spain,0,39,7,167557.12,1,-1,1,41183.02,-1
7403,796,Spain,0,56,6,94231.13,1,1,-1,121164.60,1
437,516,France,0,35,10,104088.59,2,1,-1,119666.00,-1
9837,750,France,0,38,5,151532.40,1,-1,1,46555.15,-1
2130,704,France,1,45,6,0.00,1,-1,1,137739.45,-1
7182,692,Germany,0,49,6,110540.43,2,1,1,107472.99,-1
8525,712,France,1,29,8,140170.61,1,-1,1,38170.04,-1
4510,611,Germany,1,43,9,127216.31,2,1,1,17913.25,-1


In [184]:
#Not sure if I should use this, since im doing feature creation next on these two unscaled variables
scaler = StandardScaler()
starter_df[['ScaledBalance', 'ScaledEstimatedSalary']] = scaler.fit_transform(starter_df[['Balance', 'EstimatedSalary']])

In [185]:
#use feature creation to make a ratio between age and tenure
starter_df['BalanceSalaryRatio'] = (starter_df['Balance'] / starter_df['EstimatedSalary']).round(2)
starter_df.drop(columns=['Balance', 'EstimatedSalary'], inplace=True)

In [186]:
# Create a OneHotEncoder instance
#enc = OneHotEncoder(sparse=False)

# Encode categorcal variables using OneHotEncoder
#encoded_data = enc.fit_transform(starter_df[['Geography']])

In [187]:
#encoded_df_rough = pd.DataFrame(
#encoded_data,
#columns = enc.get_feature_names(['Geography']))

In [188]:
#use feature creation to make a ratio between age and tenure
starter_df['TenureByAge'] = (starter_df['Tenure'] / starter_df['Age']).round(2)
starter_df.drop(columns=['Tenure', 'Age'], inplace=True)

In [189]:
#Remove geography. We are just doing France.
starter_df = starter_df[starter_df.Geography != 'Spain']
starter_df = starter_df[starter_df.Geography != 'Germany']
starter_df.drop(columns=['Geography'], inplace=True)

In [190]:
"""
encoded_df = pd.concat(
    [
        encoded_df_rough,
        starter_df
    ],
    axis=1
)
encoded_df.sample(10)
"""

'\nencoded_df = pd.concat(\n    [\n        encoded_df_rough,\n        starter_df\n    ],\n    axis=1\n)\nencoded_df.sample(10)\n'

In [191]:
X = starter_df.drop(columns= 'Exited')

In [192]:
y = starter_df['Exited']

In [193]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [194]:
starter_df['Exited'].value_counts()

-1    4204
 1     810
Name: Exited, dtype: int64

In [195]:
#Use SMOTE to add synthetic data and balance our target feature value count
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [196]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [197]:
#Choose a model
#clf = SVC()
#clf = LogisticRegression()
clf = RandomForestClassifier(n_estimators= 2000)

In [198]:
#Fit the model on the training data
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=2000)

In [199]:
#Make predictions on the test data
test_predictions = clf.predict(X_test)

In [200]:
accuracy = accuracy_score(test_predictions, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 79.82%


In [201]:
#Create a dataframe showing predictions vs test
comparison = pd.DataFrame({"Testing Data Predictions": test_predictions,
    "Testing Data Actual Targets": y_test})
comparison = comparison.sort_index(ascending=True)
comparison.tail(5)

,Testing Data Predictions,Testing Data Actual Targets
9972,1,-1
9977,-1,-1
9983,-1,-1
9985,1,-1
9996,-1,-1


In [202]:
#Generate a test matrix
test_matrix = confusion_matrix(y_test, test_predictions)
print(test_matrix)

[[921 125]
 [128  80]]


In [203]:
# Create a testing classifiction report
testing_report = classification_report(y_test, test_predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

          -1       0.88      0.88      0.88      1046
           1       0.39      0.38      0.39       208

    accuracy                           0.80      1254
   macro avg       0.63      0.63      0.63      1254
weighted avg       0.80      0.80      0.80      1254

